In [1]:
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split

import sys

# In case you run this without installing the package, you need to add the path to the package

# This is for launching from root folder path
sys.path.append('./ex_fuzzy/')
sys.path.append('./ex_fuzzy/ex_fuzzy/')

# This is for launching from Demos folder
sys.path.append('../ex_fuzzy/')
sys.path.append('../ex_fuzzy/ex_fuzzy/')


import ex_fuzzy.fuzzy_sets as fs
import ex_fuzzy.evolutionary_fit as GA
import ex_fuzzy.utils as  utils
import ex_fuzzy.eval_tools as eval_tools
import ex_fuzzy.persistence as persistence
import ex_fuzzy.vis_rules as vis_rules

In [2]:
runner = 1 # 1: single thread, 2+: corresponding multi-thread

n_gen = 50
n_pop = 30
    
nRules = 15
nAnts = 4
vl = 3
tolerance = 0.01
fz_type_studied = fs.FUZZY_SETS.t1

In [3]:
# Import some data to play with
iris = datasets.load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [4]:
# Compute the fuzzy partitions using 3 quartiles
precomputed_partitions = utils.construct_partitions(X, fz_type_studied)

In [5]:
# We create a FRBC with the precomputed partitions and the specified fuzzy set type, 
fl_classifier = GA.BaseFuzzyRulesClassifier(nRules=nRules, linguistic_variables=precomputed_partitions, nAnts=nAnts, 
                                            n_linguist_variables=vl, fuzzy_type=fz_type_studied, verbose=True, tolerance=tolerance, runner=runner)
fl_classifier.fit(X_train, y_train, n_gen=n_gen, pop_size=n_pop, checkpoints=0)

# Evaluate the performance of the rule base
eval_tools.eval_fuzzy_model(fl_classifier, X_train, y_train, X_test, y_test, 
                        plot_rules=False, print_rules=True, plot_partitions=False)

n_gen  |  n_eval  |     f_avg     |     f_min    
2      | 30       | 0.8233168117945705 | 0.4955324267302951
3      | 60       | 0.6808455152592435 | 0.4955324267302951
4      | 90       | 0.605200605895699 | 0.4955324267302951
5      | 120      | 0.5567202219343628 | 0.4955324267302951
6      | 150      | 0.49964153485309215 | 0.4955324267302951
7      | 180      | 0.495532426730295 | 0.4955324267302951
8      | 210      | 0.4910801842943531 | 0.36196515365203763
9      | 240      | 0.48467367871154965 | 0.36196515365203763
10     | 270      | 0.47131695140372387 | 0.36196515365203763
11     | 300      | 0.4381969912052689 | 0.36196515365203763
12     | 330      | 0.3612556887014627 | 0.3406812051347837
13     | 360      | 0.3605462237508875 | 0.3406812051347837
14     | 390      | 0.36046710022694306 | 0.3406812051347837
15     | 420      | 0.35818045832732864 | 0.3406812051347837
16     | 450      | 0.3522673681508941 | 0.3406812051347837
17     | 480      | 0.3406812051347837 | 0.

In [6]:
# Use the rule base as a candidate to further optimize the rules
frbc = fl_classifier.rule_base

refined_classifier = GA.BaseFuzzyRulesClassifier(verbose=False, tolerance=tolerance, runner=runner, linguistic_variables=None)
refined_classifier.fit(X_train, y_train, n_gen=n_gen, pop_size=n_pop, checkpoints=0, initial_rules=frbc)

# Evaluate the performance of the rule base
eval_tools.eval_fuzzy_model(fl_classifier, X_train, y_train, X_test, y_test, 
                        plot_rules=False, print_rules=True, plot_partitions=False)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       30 |  0.2406390709 |  0.2406390709
     2 |       60 |  0.2406390709 |  0.2406390709
     3 |       90 |  0.2406390709 |  0.2406390709
     4 |      120 |  0.2406390709 |  0.2406390709
     5 |      150 |  0.2392824628 |  0.1999408293
     6 |      180 |  0.2359984741 |  0.1999408293
     7 |      210 |  0.2220493366 |  0.1369396206
     8 |      240 |  0.1932673935 |  0.1369396206
     9 |      270 |  0.1771339292 |  0.1369396206
    10 |      300 |  0.1471977512 |  0.1369396206
    11 |      330 |  0.1369791618 |  0.1369396206
    12 |      360 |  0.1369396206 |  0.1369396206
    13 |      390 |  0.1369396206 |  0.1369396206
    14 |      420 |  0.1369396206 |  0.1369396206
    15 |      450 |  0.1369396206 |  0.1369396206
    16 |      480 |  0.1369396206 |  0.1369396206
    17 |      510 |  0.1369396206 |  0.1369396206
    18 |      540 |  0.1369396206 |  0.1369396206
    19 |      570 |  0.1369396206 |  0.1369396206
